In [1]:
from flask import Flask, render_template, request, jsonify,url_for
import openai
import pandas as pd
import os
from langchain.agents.agent_types import AgentType
from langchain.agents import load_tools
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_community.chat_models import ChatOpenAI
from langchain.agents.agent_types import *
from langchain_openai import OpenAI
import regex as re
import base64
import shutil
import time

app = Flask(__name__)
openai.api_key = os.environ.get('OPENAI_API_KEY')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/process', methods=['POST'])
def process():
    start_time=time.time()
    question = request.form.get('question')
    data_source = request.form.get('data_source')
    # print(question)
    print(data_source)
    
    df_BLK = pd.read_csv('C:\CODE\Demo\BLk_DATASET.csv')
    df_blk_meta=pd.read_csv('C:\CODE\Demo\df_blk_meta.csv')
    df_dtc_data=pd.read_csv('C:\CODE\Demo\dtc_dataset.csv')
    
    agent_blk_meta = create_pandas_dataframe_agent(OpenAI(temperature=0), df_blk_meta, verbose=True)

    agent_blk_meta = create_pandas_dataframe_agent(
        ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        df_blk_meta,
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS,
    )

    agent_blk = create_pandas_dataframe_agent(OpenAI(temperature=0), df_BLK, verbose=True)

    agent_blk = create_pandas_dataframe_agent(
        ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        df_BLK,
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS,
    )
    
    agent_dtc_data= create_pandas_dataframe_agent(OpenAI(temperature=0), df_dtc_data, verbose=True)

    agent_dtc_data= create_pandas_dataframe_agent(
        ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        df_dtc_data,
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS,
    )
    
    if data_source == 'metadata':
 
        df_a=agent_blk_meta.run(question)
        print(question)
        # print(df_a)

    elif data_source == 'blk':
        
        df_a=agent_blk.run(question)
        print(question)
    elif data_source == 'dtc':
        
        df_a=agent_dtc_data.run(question)

    else:
        print("Please go back and select a source from the drop down")
    
    print("#########df_a#####\n",df_a)
    
    match = re.search(r'\b[\w-]+\.(?:png|jpg|jpeg|gif)\b', df_a)
    print("here is the match if found:",match)
    
    image_url=None
    if match:
        filename = match.group()
        source_file = r'C:\CODE\Demo\static\{}'.format(filename)  # Using the filename variable
        shutil.move(r'C:\CODE\Demo\{}'.format(filename),r'C:\CODE\Demo\static\collectionimagess')
        print('**********************************')
        print(source_file)
        image_url =url_for('static',filename= r'collectionimagess/'+filename)
    print(image_url)
    end_time=time.time()
    elapsed_time = end_time - start_time
    print(f"Time consumed for processing request: {elapsed_time} seconds")
    

    return jsonify({'response': df_a,'image_path':image_url})

if __name__ == '__main__':
    app.run(debug=False)


c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [30/Apr/2024 09:55:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2024 09:55:42] "GET /static/logo.jpg HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2024 09:55:42] "GET /favicon.ico HTTP/1.1" 404 -


metadata


c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


...

Apologies, but I am unable to access the data file 'data.csv' to generate the plot. Please make sure the file is in the correct location and try again.

> Finished chain.
what is the average speed all class? give me in png format?
#########df_a#####
 Apologies, but I am unable to access the data file 'data.csv' to generate the plot. Please make sure the file is in the correct location and try again.
here is the match if found: None
None
Time consumed for processing request: 9.078015327453613 seconds
blk


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['category']=='ICE']['speed'].mean()"}`


49.34230066637761

127.0.0.1 - - [30/Apr/2024 09:59:27] "POST /process HTTP/1.1" 200 -


The average speed of all ICE class vehicles is approximately 49.34.

> Finished chain.
what is the average speed all class?
#########df_a#####
 The average speed of all ICE class vehicles is approximately 49.34.
here is the match if found: None
None
Time consumed for processing request: 10.007988929748535 seconds
blk


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import matplotlib.pyplot as plt\n\nspeed_avg = df.groupby('model_series')['speed'].mean()\n\nplt.figure(figsize=(10, 6))\nplt.bar(speed_avg.index, speed_avg.values)\nplt.xlabel('Model Series')\nplt.ylabel('Average Speed')\nplt.title('Average Speed by Model Series')\nplt.xticks(rotation=45)\nplt.savefig('average_speed.png')\nplt.close()"}`




[2024-04-30 10:00:35,942] ERROR in app: Exception on /process [POST]
Traceback (most recent call last):
  File "c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\shutil.py", line 825, in move
    os.rename(src, real_dst)
FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:\\CODE\\Demo\\average_speed.png' -> 'C:\\CODE\\Demo\\static\\collectionimagess\\average_speed.png'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\site-packages\flask\app.py", line 1823, in full_dispatch

Here is the average speed for each model series plotted in a bar chart:

![average_speed.png](average_speed.png)

> Finished chain.
what is the average speed all class? give me in png format?
#########df_a#####
 Here is the average speed for each model series plotted in a bar chart:

![average_speed.png](average_speed.png)
here is the match if found: <regex.Match object; span=(75, 92), match='average_speed.png'>
